In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
raw_panel = pd.read_csv("panel.csv")
num_teams = len(raw_panel)
raw_panel = raw_panel.to_dict('index')
panel_records = {i : [0, 0, 0] for i in range(num_teams)}


In [3]:
rooms = ["6th Grade", "5-8 Science", "4th Grade", "5th Grade", "Library", "7th Grade", "8th Grade"]
def process_ballot_info(winner, loser, winner_speaks, loser_speaks):
  previous = panel_records
  panel_records[winner][0] += 1
  panel_records[loser][1] += 1
  panel_records[winner][2] += winner_speaks
  panel_records[loser][2] += loser_speaks
  return previous

In [4]:
blocked = {}
codes = {}

for key in raw_panel:
  name = ""
  p_1 = raw_panel[key]['Partner 1'].split()
  p_2 = raw_panel[key]['Partner 2'].split()
  name += p_1[-1] + "-" + p_2[-1]
  p_3 = raw_panel[key]['Partner 3']
  if isinstance(p_3, str):
    p_3 = p_3.split()
    name += "-" + p_3[-1]
  codes[key] = name

  blocked[key] = set()
  school = raw_panel[key]['School']
  for other_key in raw_panel:
    if key == other_key: continue
    if school == raw_panel[other_key]['School']:
      blocked[key].add(other_key)

round_1 = []
neg = {i+7 for i in range(int(num_teams/2))}
aff = [i for i in range(int(num_teams/2))]
random.shuffle(aff)
room_index = 0
for aff_team in aff:
  for neg_team in neg:
    if neg_team not in blocked[aff_team]:
      round_1.append(rooms[room_index] + ": " + codes[aff_team] + " (AFF) versus " + codes[neg_team] + " (NEG)")
      blocked[aff_team].add(neg_team)
      blocked[neg_team].add(aff_team)
      neg.remove(neg_team)
      break
  room_index +=1

round_2 = []
neg = {i for i in range(int(num_teams/2))}
aff = [i + 7 for i in range(int(num_teams/2))]
random.shuffle(aff)
room_index = 0
for aff_team in aff:
  for neg_team in neg:
    if neg_team not in blocked[aff_team]:
      round_2.append(rooms[room_index] + ": " + codes[aff_team] + " (AFF) versus " + codes[neg_team] + " (NEG)")
      blocked[aff_team].add(neg_team)
      blocked[neg_team].add(aff_team)
      neg.remove(neg_team)
      break
  room_index +=1

round_3 = []
neg = {i+7 for i in range(int(num_teams/2))}
aff = [i for i in range(int(num_teams/2))]
random.shuffle(aff)
room_index = 0
for aff_team in aff:
  for neg_team in neg:
    if neg_team not in blocked[aff_team]:
      round_3.append(rooms[room_index] + ": " + codes[aff_team] + " (AFF) versus " + codes[neg_team] + " (NEG)")
      blocked[aff_team].add(neg_team)
      blocked[neg_team].add(aff_team)
      neg.remove(neg_team)
      break
  room_index +=1

with open('prelims.txt', 'w') as f:
  # round 1
  f.write('Panel Debate\n\nRound 1\n\nStart Time: 8:50 AM\n\n\n')
  for elem in round_1:
    f.write(elem+'\n\n')
  f.write('\nPanel Debate Round 2\nStart Time: 10:40 AM\n\n')
  for elem in round_2:
    f.write(elem+'\n\n')
  f.write('\nPanel Debate Round 3\nStart Time: 11:40 AM\n\n')
  for elem in round_3:
    f.write(elem+'\n\n')
  f.close()
with open('codes.txt', 'w') as f:
  for team in codes:
    f.write(str(team) + " <——— " + codes[team] + '\n\n')
  f.close()

BELOW THIS IS FOR INGESTING BALLOTS

In [5]:
# Display current situation of ballots in clear form
def get_current_sich():
  combo = {}
  for key in panel_records:
    combo[key] = [codes[key]] + panel_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name","Wins", "Losses", "Speaks"])
  print(df)

In [6]:
def ingest_ballot():
  print("Index of Team that won the round:")
  winner = int(input())
  print("Winning Team Speaker Points:")
  winner_speaks = int(input())
  print("Index of Team that lost the round:")
  loser = int(input())
  print("Losing Team Speaker Points:")
  loser_speaks = int(input())
  return process_ballot_info(winner, loser, winner_speaks, loser_speaks)


In [8]:
previous = ingest_ballot()
get_current_sich()

                      Name  Wins  Losses  Speaks
0           Cassie-Eugenia     0       0       0
1              Anna-Lucile     0       0       0
2        Emma-Lucille-Cora     0       0       0
3        Melissa-Selma-Ida     0       0       0
4       Lyda-Rosalie-Laura     0       0       0
5          Margaret-Kittie     0       0       0
6                Ida-Lilla     0       0       0
7              Alice-Angie     0       0       0
8               Bertha-Iva     0       0       0
9   Mary-Georgiana-Bridget     0       0       0
10        Annie-Lila-Lilla     0       0       0
11     Emma-Regina-Malinda     0       0       0
12             Ella-Evelyn     0       0       0
13     Florence-Wilhelmina     0       0       0


In [136]:
# Only run if error in input!

# panel_records = previous
# This is hard coded for testing
# panel_records = {0: [1, 2, 5], 1: [1, 2, 3], 2: [2, 1, 10], 3: [2, 1, 11], 4: [3, 0, 20], 5: [3, 0, 22], 6: [0, 3, 1], 7: [2, 1, 11], 8: [1, 2, 20], 9: [2, 1, 8], 10: [2, 1, 15], 11: [0, 3, 1], 12: [2, 1, 18], 13: [1, 2, 3]}

None


BELOW THIS IS FOR QUARTERFINALS

In [43]:
# calculate quarterfinals
quarters_rooms = ["5th Grade", "Library", "7th Grade", "8th Grade"]

three_win = []
two_win = []
one_win = []
for team in panel_records:
  wins = panel_records[team][0]
  speaks = panel_records[team][2]
  if wins == 3:
    three_win.append((team, speaks))
  elif wins == 2:
    two_win.append((team, speaks))
  elif wins == 1:
    one_win.append((team, speaks))
three_win = sorted(three_win, key=lambda x: x[1], reverse=True)
two_win = sorted(two_win, key=lambda x: x[1], reverse=True)
one_win = sorted(one_win, key=lambda x: x[1], reverse=True)
temp = three_win + two_win + one_win
temp = temp[:8]
seeds = []
for i in range(len(temp)):
  seeds.append(temp[i][0])
quarters = []
room_index = 0
for i in range(int(len(seeds)/2)):
  quarters.append(quarters_rooms[room_index] + ": " + codes[seeds[i]] + " versus " + codes[seeds[-i-1]] + " (FLIP FOR SIDES)")
  room_index +=1

with open('quarters.txt', 'w') as f:
  
  f.write('Panel Debate\n\nQuarterfinals\n\nStart Time: 2:00 PM\n\n\n')
  for elem in quarters:
    f.write(elem+'\n\n')
  f.close()

